## Lisence

In [35]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Modifications copyright 2020 Dennis Adelved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

## Imports

In [36]:
import glob
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import pandas as pd

import xml.etree.ElementTree as ET
import copy
import shutil

import re

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  #raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


# This is needed to display the images.
%matplotlib inline
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Path Configuration

In [80]:
#Set the path to the folder containing the exported inference graph
MODEL_NAME = 'graph_plugs_fine_tune'

#adding the frozen inference graph to the path
PATH_TO_FROZEN_GRAPH = os.path.join(MODEL_NAME,'frozen_inference_graph.pb')

#Path to the label map
PATH_TO_LABELS = 'labelmap_plugs.pbtxt'

#Path to the images that are used for inference. Here only three sample images are given.
#However, more can be added to this directory
PATH_TO_TEST_IMAGES_DIR = 'images'

#The output directory for the cropped core images. If no cropping is desired set OUTPUT_DIR = None
OUTPUT_DIR = os.path.join('autolabel')

## Load a (frozen) Tensorflow model into memory.

In [62]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [39]:
'''category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)'''

NameError: name 'label_map_util' is not defined

In [41]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [81]:
#Run inference on image using the frozen inference graph
def run_inference_for_single_image(image, graph):


    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)


    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
    # Run inference
    output_dict = sess.run(tensor_dict,
                         feed_dict={image_tensor: image})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
      'detection_classes'][0].astype(np.int64)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [115]:
#Generate list of image paths expected input : RGB image with dimensions [height,width,depth]



TEST_IMAGE_PATHS = []
for im in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    if '.jpg' in im:
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR + '/' +  im))

#empty list for saving the output dicts for each prediction
dicts = []
IMAGE_SIZE = (20,20)
#Retrieve the detection data from the inference
with detection_graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        for op in ops: 
            op._set_device('/device:CPU:*')
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)

        for image_path in TEST_IMAGE_PATHS:
            
            #Convert RGB image to grayscale
            image = Image.open(image_path).convert('L')

            img = np.asarray(image)
            new_im = np.ndarray((img.shape[0],img.shape[1],3))
            for d in range(new_im.shape[-1]):
                new_im[:,:,d] = img[:,:]
                
            image_np = np.copy(new_im)



            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)

            # Actual detection.
            output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)  
            
            #append detection from image to the detection list
            dicts.append(output_dict)
            

## Post-Processing Functions

In [113]:

def sorted_boxes(denrom,classes,sortby=0):
    concat = np.hstack((denorm,classes)).astype(int)
    sort_val = np.zeros_like(concat)
    sortind = np.argsort(denorm[:,0].argsort())
    for i in range(sortind.shape[0]):
        row = np.argwhere(sortind == i)
        sort_val[i,:] = concat[row,:]
    return sort_val

def pixel_to_depth(top,base,pixel_height):
    return (base-top) / pixel_height

def plug_to_depth(impath,boxes):
    im = np.asarray(Image.open(impath))
    top,base = float(impath.split('.')[0].split('_')[-2]), float(impath.split('.')[0].split('_')[-1])
    meter_per_pixel = pixel_to_depth(top,base,im.shape[0])
    mid_diff = ((boxes[:,2] - boxes[:,0]) / 2)
    mid = mid_diff + boxes[:,0]
    names = []
    depth = []
    pix = []
    image = []
    for i in range(len(mid)):

        names.append(class_to_name(boxes[i,-1]))
        depth.append(mid[i] * meter_per_pixel + top)
        pix.append(mid[i])
        image.append(impath)
        
    d = {'PlugType': names, 'Depth' : depth,'Pixel Location' : pix,'Source':image}
    df = pd.DataFrame(d)
    return df


#Denormalize the bounding box coordinates generated by the model
def denormalize(array,image):
    if array.ndim ==1:
        array = array[None,:]
    print(array.shape)
    denorm = np.zeros_like(array)
    h,w = image.shape[0:2]
    denorm[:,0] = array[:,0] * h
    denorm[:,1] = array[:,1] * w
    denorm[:,2] = array[:,2] * h
    denorm[:,3] = array[:,3] * w
    return denorm

#non-max supression function (remove overlaping bounding boxes)
def non_max_supression(boxes,overlapThresh):
    if len(boxes) == 0:
        return []
    
    pick = []
    xmin = boxes[:,0]
    ymin = boxes[:,1]
    xmax = boxes[:,2]
    ymax = boxes[:,3]
    area = (xmax - xmin + 1) * (ymax - ymin + 1)
    idxs = np.argsort(ymax)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        xxmin = np.maximum(xmin[i], xmin[idxs[:last]])
        yymin = np.maximum(ymin[i], ymin[idxs[:last]])
        xxmax = np.minimum(xmax[i], xmax[idxs[:last]])
        yymax = np.minimum(ymax[i], ymax[idxs[:last]])
        
        w = np.maximum(0, xxmax - xxmin + 1)
        h = np.maximum(0, yymax - yymin + 1)
 

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    return boxes[pick].astype("int")
    
#returns the filename and file extension from the path
def get_filename(path):
    filename = re.findall('^(.+)/([^/]+)$',path)
    filename = filename[0][1].split('.')
    return filename


def query_image_from_measurement(measurement_depth,dataframe,pad=80):
    ind = (np.abs(dataframe['Depth'].values - measurement_depth)).argmin()
    center = dataframe['Pixel Location'].iloc[ind]
    testim = np.asarray(Image.open(dataframe['Source'][ind]))
    crop = testim[int(center)-pad:int(center)+pad]
    return crop



def name_from_path(image_path):
    name_components = image_path.split('/')[-1].split('_')[0:3]
    name = name_components[0] + '_' + name_components[1] + '-' + name_components[2]
    top = (image_path.split('_')[-2])
    base = (image_path.split('_')[-1].split('.jpg')[0])
    return name,float(top),float(base)

def get_well(image_path,df):
    name,top,base = name_from_path(image_path)
    current_well = df[df['Well Name'] == name]
    g_top =  current_well[current_well['Measured Depth'] >= top]
    data = g_top[g_top['Measured Depth'] <= base]
    
    return data



    

def depth_conversion(df,plugtype):
    df = df[df['PlugType'] == plugtype].values
    pixel_diff = np.diff(df[:,2])
    depth_diff = np.diff(df[:,1])
    return depth_diff/pixel_diff



def get_well_log_demo():    
    wellpath = glob.glob(os.path.join('wellog','*'))
    data = []
    header = []
    head_ind = 0
    val_ind = 0
    with open(wellpath[0]) as f:
        for ind,line in enumerate(f):
            data.append(line)
            if '~Curves' in line:
                head_ind = ind + 1
            if '~Ascii' in line:
                val_ind = ind + 1
    data = [e.split() for e in data[val_ind:]]

    return data




def class_to_name(c):
    if c == 1:
        name = 'hplug'
        return name
    if c == 2:
        name = 'vplug'
        return name
    if c == 3: 
        name = 'scal'
        return name
    else:
        return 'Non-defined class'
    



def match_objects(in_xml,in_prediction):
    counter = 0
    new = []
    while counter < len(in_prediction):
        temp = copy.deepcopy(in_xml[0])
        new.append(temp)
        counter += 1
    return new





def auto_annotate_xml(im_path,xml_template_path,predicted_bounding_boxes):
    
    output_path = os.path.join(os.getcwd(),'autolabel')
    new_xml_path = im_path.split('/')[-1].split('.')[0] + '.xml'
    xml_copy_path = shutil.copy(xml_template_path, os.path.join(output_path, new_xml_path))
    print(xml_copy_path)
    
    tree = ET.parse(xml_copy_path)
    root = tree.getroot()
    
   

    
    
    
    im = Image.open(im_path)
    im = np.asarray(im)
    h,w = im.shape
    d = 1
    root.find('folder').text = output_path.split('/')[-1]
    root.find('filename').text = im_path.split('/')[-1]
    root.find('path').text = os.path.join(output_path,im_path.split('/')[-1])
    root.find('size').find('width').text = str(w)
    root.find('size').find('height').text = str(h)
    root.find('size').find('depth').text = str(d)
    
    new_objects = match_objects(root.findall('object'),predicted_bounding_boxes)
    
    
    for obj in root.findall('object'):
        root.remove(obj)
    #print(len(root.findall('object')))
    
    
        
    for new_obj in new_objects:
        root.append(new_obj)
    #print(len(root.findall('object')))
        
    #print(len(root.findall('object')))
    #print(len(denorm))
    for ind,obj in enumerate(root.findall('object')):

        obj.find('name').text = class_to_name(predicted_bounding_boxes[ind,-1])
        obj.find('bndbox').find('ymin').text = str((predicted_bounding_boxes[ind,0]))
        obj.find('bndbox').find('xmin').text = str((predicted_bounding_boxes[ind,1]))
        obj.find('bndbox').find('ymax').text =str( (predicted_bounding_boxes[ind,2]))
        obj.find('bndbox').find('xmax').text = str((predicted_bounding_boxes[ind,3]))

    
    
    shutil.copy(im_path,os.path.join(output_path,im_path.split('/')[-1]))
    tree.write(xml_copy_path)
    

In [114]:
#Make xml files from predictions and export to output directory
for ind,d in enumerate(dicts):
    s= np.argwhere(d['detection_scores'] > 0.5)
    boxes = np.squeeze(d['detection_boxes'][s])
    classes = d['detection_classes'][s]
    image_np = Image.open(TEST_IMAGE_PATHS[ind])
    image_np = np.asarray(image_np)
    denorm = denormalize(boxes,image_np)
    denorm = sorted_boxes(denorm,classes)
    
    
    auto_annotate_xml(TEST_IMAGE_PATHS[ind],'template.xml',denorm)
    
    
    

(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3755_3756.xml
(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3757_3758.xml
(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3753_3754.xml
(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3752_3753.xml
(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3751_3752.xml
(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3716_3717.xml
(3, 4)
/home/dennis/anaconda3/envs/gputest/models/research/object_detection_latest_gpu/Use_case/autolabel/autolabel/6407_1_3_3750_3751.xml
(3, 4)
/home/dennis/anacond